In [1]:
# !pip install fastapi uvicorn mysql-connector-python python-dotenv
# !pip install watchgod

In [ ]:
import requests

response = requests.get("http://localhost:3001/api/sales?year=2023")
if response.status_code == 200:
    print("Monthly Sales:", response.json())
else:
    print("Error:", response.status_code, response.text)


In [2]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()

app = FastAPI()

# Enable CORS (adjust origins as needed)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, use the specific domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Database connection
def get_connection():
    return mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME"),
    )

@app.get("/api/sales")
def get_sales_by_year(year: int):
    try:
        conn = get_connection()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT 
              MONTH(date) AS month,
              SUM(total_earn) AS total_earn
            FROM sales
            WHERE YEAR(date) = %s
            GROUP BY MONTH(date)
            ORDER BY MONTH(date);
        """
        cursor.execute(query, (year,))
        results = cursor.fetchall()

        # Fill missing months with 0
        monthly_totals = [0.0] * 12
        for row in results:
            monthly_totals[row["month"] - 1] = float(row["total_earn"])

        return monthly_totals

    except mysql.connector.Error as err:
        print("MySQL Error:", err)
        raise HTTPException(status_code=500, detail="Database error")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [ ]:
from fastapi import FastAPI, HTTPException
from typing import List
import mysql.connector
from mysql.connector import errorcode
from pydantic import BaseModel
import os

app = FastAPI()

# Load your RDS connection info from environment variables or config
RDS_HOST = os.getenv("RDS_HOST", "your-rds-host")
RDS_USER = os.getenv("RDS_USER", "your-username")
RDS_PASSWORD = os.getenv("RDS_PASSWORD", "your-password")
RDS_DB = os.getenv("RDS_DB", "your-database")

class ProductStock(BaseModel):
    ProductID: str
    Balance: int

def get_db_connection():
    try:
        conn = mysql.connector.connect(
            host=RDS_HOST,
            user=RDS_USER,
            password=RDS_PASSWORD,
            database=RDS_DB
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to DB: {err}")
        return None

@app.get("/inventory/{store_id}", response_model=List[ProductStock])
def get_inventory(store_id: str):
    conn = get_db_connection()
    if not conn:
        raise HTTPException(status_code=500, detail="Database connection error")

    cursor = conn.cursor(dictionary=True)

    # Query to get latest date per store, and get all products' balances for that date
    # Adjust table/column names to your actual schema
    query = """
        SELECT ProductID, Balance FROM inventory_data
        WHERE StoreID = %s
          AND Date = (
            SELECT MAX(Date) FROM inventory_data WHERE StoreID = %s
          )
        ORDER BY ProductID
    """

    cursor.execute(query, (store_id, store_id))
    results = cursor.fetchall()
    cursor.close()
    conn.close()

    # Return list of dict {ProductID, Balance}
    return [ProductStock(**row) for row in results]


FileNotFoundError: [Errno 2] No such file or directory: 'inventory_data.csv'

In [16]:
import requests

BASE_URL = "http://127.0.0.1:3333"

def test_sales(year: int):
    try:
        response = requests.get(f"{BASE_URL}/api/sales", params={"year": year})
        response.raise_for_status()
        print(f"\n✅ Sales Data for {year}:")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching sales data: {e}")

def test_inventory():
    try:
        response = requests.get(f"{BASE_URL}/api/inventory")
        response.raise_for_status()
        print("\n✅ Inventory Data:")
        for record in response.json():
            print(record)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching inventory data: {e}")
        
        
def test_tables():
    try:
        response = requests.get(f"{BASE_URL}/api/tables")
        response.raise_for_status()
        print("\n✅ Tables:")
        for record in response.json():
            print(record)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching inventory data: {e}")


In [24]:
def test_tables():
    try:
        response = requests.get(f"http://127.0.0.1:3333/api/tables")
        response.raise_for_status()
        print("\n✅ Tables:")
        for record in response.json():
            print(record)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching inventory data: {e}")
        
table_infos = test_tables()

❌ Error fetching inventory data: 500 Server Error: Internal Server Error for url: http://127.0.0.1:3333/api/tables


In [25]:
test_sales(2024)


✅ Sales Data for 2024:
[50343.2, 205084.19999999995, 327273.0000000001, 424259.6000000001, 205477.40000000002, 389137.60000000015, 326802.79999999993, 375692.39999999985, 397557.3999999999, 491170.3999999999, 385433.59999999986, 339866.4000000001]


In [13]:
# import requests

# response = requests.get("http://localhost:3333/api/inventory")
# if response.status_code == 200:
#     inventory = response.json()
#     print(inventory)
# else:
#     print("Failed to fetch inventory:", response.status_code)

In [5]:
import requests

response = requests.get("http://localhost:3334/inventoryall")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)

[{'inventoryID': 'INV001', 'productID': 'PRD001', 'CurrentStock': 120.0, 'LeadTime': 7.0, 'ReorderPoint': 28.0, 'SafetyStock': 10.0, 'PT': '20250517235610'}, {'inventoryID': 'INV002', 'productID': 'PRD002', 'CurrentStock': 150.0, 'LeadTime': 7.0, 'ReorderPoint': 32.0, 'SafetyStock': 12.0, 'PT': '20250517235610'}, {'inventoryID': 'INV003', 'productID': 'PRD003', 'CurrentStock': 130.0, 'LeadTime': 7.0, 'ReorderPoint': 26.0, 'SafetyStock': 9.0, 'PT': '20250517235610'}, {'inventoryID': 'INV004', 'productID': 'PRD004', 'CurrentStock': 100.0, 'LeadTime': 10.0, 'ReorderPoint': 35.0, 'SafetyStock': 14.0, 'PT': '20250517235610'}, {'inventoryID': 'INV005', 'productID': 'PRD005', 'CurrentStock': 90.0, 'LeadTime': 14.0, 'ReorderPoint': 40.0, 'SafetyStock': 16.0, 'PT': '20250517235610'}, {'inventoryID': 'INV006', 'productID': 'PRD006', 'CurrentStock': 80.0, 'LeadTime': 14.0, 'ReorderPoint': 38.0, 'SafetyStock': 15.0, 'PT': '20250517235610'}]


In [ ]:
import requests

response = requests.get("http://localhost:3334/salesall")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)
    
#Sales: salesID, userID, order_date, amount

[{'salesID': 'S001', 'userID': 'U032', 'order_date': '2025-01-01T00:00:00', 'amount': 99.8, 'PT': '20250517214514'}, {'salesID': 'S002', 'userID': 'U017', 'order_date': '2025-01-01T00:00:00', 'amount': 89.8, 'PT': '20250517214514'}, {'salesID': 'S003', 'userID': 'U042', 'order_date': '2025-01-01T00:00:00', 'amount': 139.7, 'PT': '20250517214514'}, {'salesID': 'S004', 'userID': 'U010', 'order_date': '2025-01-01T00:00:00', 'amount': 49.9, 'PT': '20250517214514'}, {'salesID': 'S005', 'userID': 'U025', 'order_date': '2025-01-01T00:00:00', 'amount': 89.8, 'PT': '20250517214514'}, {'salesID': 'S006', 'userID': 'U022', 'order_date': '2025-01-01T00:00:00', 'amount': 49.9, 'PT': '20250517214514'}, {'salesID': 'S007', 'userID': 'U016', 'order_date': '2025-01-01T00:00:00', 'amount': 39.9, 'PT': '20250517214514'}, {'salesID': 'S008', 'userID': 'U012', 'order_date': '2025-01-01T00:00:00', 'amount': 119.7, 'PT': '20250517214514'}, {'salesID': 'S009', 'userID': 'U012', 'order_date': '2025-01-01T00:00

In [ ]:
import requests

response = requests.get("http://localhost:3334/inventoryall")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)
    
#Order: orderitemID, salesID, productID, quantity, price

[{'orderitemID': 'OI001', 'salesID': 'S001', 'productID': 'PRD005', 'quantity': 1.0, 'price': 49.9, 'PT': '20250517214617'}, {'orderitemID': 'OI002', 'salesID': 'S001', 'productID': 'PRD004', 'quantity': 1.0, 'price': 49.9, 'PT': '20250517214617'}, {'orderitemID': 'OI003', 'salesID': 'S002', 'productID': 'PRD006', 'quantity': 1.0, 'price': 49.9, 'PT': '20250517214617'}, {'orderitemID': 'OI004', 'salesID': 'S002', 'productID': 'PRD003', 'quantity': 1.0, 'price': 39.9, 'PT': '20250517214617'}, {'orderitemID': 'OI005', 'salesID': 'S003', 'productID': 'PRD003', 'quantity': 1.0, 'price': 39.9, 'PT': '20250517214617'}, {'orderitemID': 'OI006', 'salesID': 'S003', 'productID': 'PRD004', 'quantity': 2.0, 'price': 49.9, 'PT': '20250517214617'}, {'orderitemID': 'OI007', 'salesID': 'S004', 'productID': 'PRD005', 'quantity': 1.0, 'price': 49.9, 'PT': '20250517214617'}, {'orderitemID': 'OI008', 'salesID': 'S005', 'productID': 'PRD004', 'quantity': 1.0, 'price': 49.9, 'PT': '20250517214617'}, {'order

In [9]:
import requests

response = requests.get("http://localhost:3334/product")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)
    
#Product: productID, product_name, description, CurrentStock, ReorderPoint, SafetyStock, LeadTime

[{'productID': 'PRD001', 'product_name': 'Mamak Series', 'description': 'Roti Canai, Teh Tarik, Maggi Goreng, Nasi Kandar', 'cost': 9.9, 'CurrentStock': 120.0, 'ReorderPoint': 28.0, 'SafetyStock': 10.0, 'LeadTime': 7.0, 'restock_required': 0}, {'productID': 'PRD002', 'product_name': 'Kopitiam Series', 'description': 'Char Kuey Teow, Wantan Mee, Dim Sum, Kopi Peng', 'cost': 9.9, 'CurrentStock': 150.0, 'ReorderPoint': 32.0, 'SafetyStock': 12.0, 'LeadTime': 7.0, 'restock_required': 0}, {'productID': 'PRD003', 'product_name': 'Warung Series', 'description': 'Nasi Lemak, Satay, Kuih Muih, Air Bandung', 'cost': 9.9, 'CurrentStock': 130.0, 'ReorderPoint': 26.0, 'SafetyStock': 9.0, 'LeadTime': 7.0, 'restock_required': 0}, {'productID': 'PRD004', 'product_name': 'Pasar Malam Series', 'description': 'Apam Balik, Lok Lok, Otak-Otak, Coconut Shake', 'cost': 9.9, 'CurrentStock': 100.0, 'ReorderPoint': 35.0, 'SafetyStock': 14.0, 'LeadTime': 10.0, 'restock_required': 0}, {'productID': 'PRD005', 'prod

In [18]:
import requests

response = requests.get("http://localhost:3334/api/tables")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)

{'tables': [['forecast'], ['forecasted'], ['inventory'], ['product'], ['qbi_file_20250517_13_04_46_0'], ['qbi_file_20250517_13_05_05_0'], ['qbi_file_20250517_21_37_08_0'], ['qbi_file_20250517_21_45_35_0'], ['sales'], ['user']]}


In [4]:
import requests

response = requests.get("http://localhost:3334/product")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)

[{'productID': 'PRD001', 'name': 'Mamak Series', 'release_date': '2024-06-01T00:00:00', 'description': 'Roti Canai, Teh Tarik, Maggi Goreng, Nasi Kandar', 'cost': 9.9, 'PT': '20250517214114'}, {'productID': 'PRD002', 'name': 'Kopitiam Series', 'release_date': '2024-06-01T00:00:00', 'description': 'Char Kuey Teow, Wantan Mee, Dim Sum, Kopi Peng', 'cost': 9.9, 'PT': '20250517214114'}, {'productID': 'PRD003', 'name': 'Warung Series', 'release_date': '2024-06-01T00:00:00', 'description': 'Nasi Lemak, Satay, Kuih Muih, Air Bandung', 'cost': 9.9, 'PT': '20250517214114'}, {'productID': 'PRD004', 'name': 'Pasar Malam Series', 'release_date': '2024-07-01T00:00:00', 'description': 'Apam Balik, Lok Lok, Otak-Otak, Coconut Shake', 'cost': 9.9, 'PT': '20250517214114'}, {'productID': 'PRD005', 'name': 'Beverage Series', 'release_date': '2024-09-01T00:00:00', 'description': 'Milo Dinosaur, Sugarcane juice, Sirap Limau, Cendol', 'cost': 9.9, 'PT': '20250517214114'}, {'productID': 'PRD006', 'name': 'Ny

In [2]:
import requests

response = requests.get("http://localhost:3334/forecastall")
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Failed to fetch:", response.status_code)

[{'productID': 'PRD006', 'days': 30.0, 'forecasted': 8.0, 'current_stock': 80.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': '20250518034218'}, {'productID': 'PRD001', 'days': 1.0, 'forecasted': 2.0, 'current_stock': 120.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD001', 'days': 3.0, 'forecasted': 6.0, 'current_stock': 120.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD001', 'days': 5.0, 'forecasted': 15.0, 'current_stock': 120.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD001', 'days': 7.0, 'forecasted': 38.0, 'current_stock': 120.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD001', 'days': 30.0, 'forecasted': 3669.0, 'current_stock': 120.0, 'stock_status': 'Restock Needed', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD002', 'days': 1.0, 'forecasted': 1.0, 'current_stock': 150.0, 'stock_status': 'OK', 'weeks': 0.0, 'PT': None}, {'productID': 'PRD002', 'days': 3.0, 'forecasted': 0.0, 'current_s

In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [17]:
import requests

url = "http://localhost:3334/api/predict"
payload = {"date": "2025-05-20"}

response = requests.post(url, json=payload)

print("Status Code:", response.status_code)
print("Response Text:", response.text)

try:
    data = response.json()
    print("JSON Response:", data)
except requests.exceptions.JSONDecodeError:
    print("Failed to decode JSON. Raw response:")
    print(response.text)


Status Code: 200
Response Text: {"requested_date":"2025-05-20","last_updated_data":"2025-05-17","predictions":[{"ProductID":"PRD001","Days":1.0,"Forecasted":2,"Current Stock":120,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD001","Days":3.0,"Forecasted":6,"Current Stock":120,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD001","Days":5.0,"Forecasted":15,"Current Stock":120,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD001","Days":7.0,"Forecasted":38,"Current Stock":120,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD001","Days":30.0,"Forecasted":3669,"Current Stock":120,"Stock Status":"Restock Needed","Weeks":0.0},{"ProductID":"PRD002","Days":1.0,"Forecasted":1,"Current Stock":150,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD002","Days":3.0,"Forecasted":-1,"Current Stock":150,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD002","Days":5.0,"Forecasted":-4,"Current Stock":150,"Stock Status":"OK","Weeks":0.0},{"ProductID":"PRD002","Days":7.0,"Forecasted":-5,"Current Sto

In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [ ]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [2]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [4]:
import requests

# URL of your FastAPI service (adjust if hosted elsewhere)
url = "http://localhost:3333/api/chatbot/"

# Sample payload
payload = {
    "prompt": "Give me inventory forecast analysis for today.",
    "inventory": {
        "Product A": 10,
        "Product B": 3,
        "Product C": 0
    },
    "lstm": {
        "Product A": 30,
        "Product B": 20,
        "Product C": 5
    },
    "date": "2025-05-17"
}

# Make the POST request
response = requests.post(url, json=payload)

# Print the result
if response.status_code == 200:
    print("✅ Response from chatbot:")
    print(response.json()["response"])
else:
    print("❌ Error:")
    print(response.status_code)
    print(response.text)

✅ Response from chatbot:
📦 Inventory Forecast Analysis for Today:

➤ **Product A**: 10 units in stock

➤ **Product B**: 3 units left — low stock!

➤ **Product C**: Out of stock with 0 units


In [22]:
import requests

response = requests.get("http://localhost:3333/api/inventory")
if response.status_code == 200:
    inventory = response.json()
    print(inventory)
else:
    print("Failed to fetch inventory:", response.status_code)

Failed to fetch inventory: 500


In [ ]:
uvicorn server:app --host 0.0.0.0 --port 3333 --reload  